In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from spacy.matcher import Matcher
import networkx as nx
import tensorflow_hub as hub
import time
from tqdm import tqdm
import math

Vectorisation

In [2]:
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.width', 120)
path = os.getcwd()

In [7]:
# =================================================================================
# LOADING CLEAN DATA
# =================================================================================
df = pickle.load(open(path+'\\cleaned_data_no_null.sav', 'rb'))
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
embed = hub.load(path+'\\model')

In [12]:
# ABSTRACT VECTORISATION
abstract_vec = np.zeros(shape=(1, 512))
for i in tqdm(range(math.ceil(len(df)/100))):
    if (i < math.ceil(len(df)/100)-1):
        my_abstract = df.loc[(i*100):((i+1)*100-1), 'abstract']
    else:
        my_abstract = df.loc[(i*100):len(df), 'abstract']
    embed_vectors = embed(my_abstract.values).numpy()
    abstract_vec = np.append(abstract_vec, embed_vectors, axis=0)

abstract_vec = np.delete(abstract_vec, 0, axis=0)

100%|████████████████████████████████████████████████████████████████████████████| 1605/1605 [3:29:53<00:00,  7.85s/it]


NameError: name 'path_data' is not defined

In [13]:
pickle.dump(abstract_vec, open(path+'\\abstract_vectors.sav', 'wb'))

Model

In [3]:
abstract_vec = pickle.load(open(path+'\\abstract_vectors.sav', 'rb'))

In [4]:
def prepare_similarity(vectors):
    similarity = cosine_similarity(vectors)
    return similarity

In [10]:
question = "best covid"
question = pd.Series(question)
question_vec = embed(question.values).numpy()
embed_vectors = np.append(abstract_vec, question_vec, axis=0)

In [11]:
# cosine TODO reduce memory usage 1. Approximate Algorithm or 2. 分步计算
similarity_matrix = prepare_similarity(embed_vectors)

MemoryError: Unable to allocate 192. GiB for an array with shape (160416, 160416) and data type float64

In [22]:
abstract_vec

array([[-2.35034648e-04, -3.98643222e-03,  4.23211940e-02, ...,
        -3.42577067e-03, -3.50324698e-02,  9.52725857e-02],
       [ 7.30029494e-02, -2.99957339e-02, -6.49087578e-02, ...,
        -2.84780376e-02, -3.87244485e-02, -7.77232880e-03],
       [ 6.58439472e-02, -5.62125146e-02, -1.77819207e-02, ...,
        -5.42196743e-02, -6.17322139e-02,  5.25257029e-02],
       ...,
       [-9.86412391e-02,  2.95698438e-02,  2.67106220e-02, ...,
         5.02402690e-05, -6.42253435e-04, -3.78320403e-02],
       [-5.78135327e-02,  1.01426229e-01, -7.36692222e-03, ...,
         7.52908885e-02, -3.21089774e-02, -1.03685353e-02],
       [-7.16842152e-03,  4.41300534e-02, -1.81317218e-02, ...,
         7.04283938e-02, -2.78379191e-02,  7.08289668e-02]])